In [1]:
library(ggplot2)

In [2]:
grid_df <- read.csv("/home/jeremy/tornadoesr/data/raw/grid_with_predictions.csv")
cities_df <- read.csv("/home/jeremy/tornadoesr/data/raw/cities_with_predictions.csv")

In [3]:
old_df <- read.csv("/home/jeremy/tornadoesr/data/raw/tor_data_inflation_adj.csv")

sd_lat <- sd(old_df$BEGIN_LAT)
sd_lon <- sd(old_df$BEGIN_LON)
mu_lat <- mean(old_df$BEGIN_LAT)
mu_lon <- mean(old_df$BEGIN_LON)

grid_df$BEGIN_LAT <- grid_df$BEGIN_LAT*sd_lat + mu_lat
grid_df$BEGIN_LON <- grid_df$BEGIN_LON*sd_lon + mu_lon
cities_df$BEGIN_LAT <- cities_df$BEGIN_LAT*sd_lat + mu_lat
cities_df$BEGIN_LON <- cities_df$BEGIN_LON*sd_lon + mu_lon

In [4]:
states <- map_data("state")

In [5]:
# mkdir gif_cond_images

In [6]:
month.name

[1] "January"   "February"  "March"     "April"     "May"       "June"     
 [7] "July"      "August"    "September" "October"   "November"  "December"

In [7]:
count <- 1
for (i in unique(cities_df$JULIAN_DAY)) {
    month <- i
    title <- month.name[count]
    t_cities_df <- dplyr::filter(cities_df, JULIAN_DAY == month)
    t_grid_df <- dplyr::filter(grid_df, JULIAN_DAY == month)
    a <- ggplot(data = states) + 
      geom_polygon(aes(x = long, y = lat, group = group), color = "white", fill = 'NA', lwd = 0.5) +
      viridis::scale_color_viridis('Conditional property\ndamage',
                                   direction = -1, option = 'B',
                                   breaks = c(200000,
                                              600000,
                                              1000000,
                                              1400000,
                                              1800000,
                                              2200000),
                                   labels = c('$200,000',
                                              '$600,000',
                                              '$1,000,000',
                                              '$1,400,000',
                                              '$1,800,000',
                                              '$2,200,000'),
                                   limits = c(0, 2200000)) + 
      coord_quickmap() +
      guides(fill = FALSE) +
      geom_point(data = t_grid_df, aes(x = BEGIN_LON,
                                           y = BEGIN_LAT,
                                           color = DAMAGE_PROPERTY),
                   size = 2) +
      geom_point(data = t_cities_df, aes(x = BEGIN_LON,
                                           y = BEGIN_LAT,
                                           color = DAMAGE_PROPERTY),
                   size = 4, pch = 22, stroke = 2) +
      theme_minimal() +
      theme(panel.grid = element_blank(),
            panel.background = element_rect(fill = "black"),
            axis.title = element_blank(),
            axis.text = element_blank(),
            axis.ticks = element_blank()) +
      labs(title = paste0('Conditional property damage prediction for: ', title, ' 15, 2019'))
    if (count < 10) {
        ggsave(paste0('gif_cond_images/0', as.character(count), '.png'), height = 8.5, width = 11, units = 'in')
    } else {
        ggsave(paste0('gif_cond_images/', as.character(count), '.png'), height = 8.5, width = 11, units = 'in')    
    }
    count <- count + 1
}

In [8]:
# cd gif_cond_images
# convert -delay 90 -loop 0 *.png conditional_map.gif
# cd ..
# mkdir gif_bin_images

In [9]:
count <- 1
for (i in unique(cities_df$JULIAN_DAY)) {
    month <- i
    title <- month.name[count]
    t_cities_df <- dplyr::filter(cities_df, JULIAN_DAY == month)
    t_grid_df <- dplyr::filter(grid_df, JULIAN_DAY == month)
    a <- ggplot(data = states) + 
      geom_polygon(aes(x = long, y = lat, group = group), color = "black", fill = 'NA', lwd = 0.5) +
      viridis::scale_color_viridis('Probability of\ndamage',
                                   direction = -1, option = 'B',
                                   breaks = c(0, 0.25, 0.5, 0.75, 1)) + 
      coord_quickmap() +
      guides(fill = FALSE) +
      geom_point(data = t_grid_df, aes(x = BEGIN_LON,
                                           y = BEGIN_LAT,
                                           color = DAMAGE_PROB),
                   size = 2) +
      geom_point(data = t_cities_df, aes(x = BEGIN_LON,
                                           y = BEGIN_LAT,
                                           color = DAMAGE_PROB),
                   size = 4, pch = 22, stroke = 2) +
      theme_minimal() +
      theme(panel.grid = element_blank(),
            axis.title = element_blank(),
            axis.text = element_blank(),
            axis.ticks = element_blank()) +
      labs(title = paste0('Probability of damage occurrence prediction for: ', title, ' 15, 2019'))
    if (count < 10) {
        ggsave(paste0('gif_bin_images/0', as.character(count), '.png'), height = 8.5, width = 11, units = 'in')
    } else {
        ggsave(paste0('gif_bin_images/', as.character(count), '.png'), height = 8.5, width = 11, units = 'in')    
    }
    count <- count + 1
}

In [10]:
# cd gif_bin_images
# convert -delay 90 -loop 0 *.png binary_map.gif